In [2]:
# importações
import pandas as pd
import numpy as np

df = pd.read_csv("C:/Users/vitor/OneDrive/Documentos/My-projects/Southampton/Dados/epl_player_stats_24_25.csv")
df


,Player Name,Club,Nationality,Position,Appearances,Minutes,Goals,Assists,Shots,Shots On Target,...,Fouls,Yellow Cards,Red Cards,Saves,Saves %,Penalties Saved,Clearances Off Line,Punches,High Claims,Goals Prevented
0,Ben White,Arsenal,England,DEF,17,1198,0,2,9,12,...,10,2,0,0,0%,0,0,0,0,0.0
1,Bukayo Saka,Arsenal,England,MID,25,1735,6,10,67,2,...,15,3,0,0,0%,0,0,0,0,0.0
2,David Raya,Arsenal,Spain,GKP,38,3420,0,0,0,0,...,1,3,0,86,72%,0,0,8,53,2.1
3,Declan Rice,Arsenal,England,MID,35,2833,4,7,48,18,...,21,5,1,0,0%,0,0,0,0,0.0
4,Ethan Nwaneri,Arsenal,England,MID,26,889,4,0,24,0,...,9,1,0,0,0%,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,Santiago Bueno,Wolverhampton Wanderers,Uruguay,DEF,29,1682,0,0,6,3,...,26,2,0,0,0%,0,0,0,0,0.0
558,Tom King,Wolverhampton Wanderers,Wales,GKP,1,1,0,0,0,0,...,0,0,0,0,0%,0,0,0,0,0.0
559,Tommy Doyle,Wolverhampton Wanderers,England,MID,24,475,0,1,9,0,...,5,1,0,0,0%,0,0,0,0,0.0
560,Toti Gomes,Wolverhampton Wanderers,Portugal,DEF,31,2615,0,0,7,8,...,24,7,0,0,0%,0,2,0,0,0.0


In [3]:
# procurando por valores NaN
print(df.isna())

     Player Name   Club  Nationality  Position  Appearances  Minutes  Goals  \
0          False  False        False     False        False    False  False   
1          False  False        False     False        False    False  False   
2          False  False        False     False        False    False  False   
3          False  False        False     False        False    False  False   
4          False  False        False     False        False    False  False   
..           ...    ...          ...       ...          ...      ...    ...   
557        False  False        False     False        False    False  False   
558        False  False        False     False        False    False  False   
559        False  False        False     False        False    False  False   
560        False  False        False     False        False    False  False   
561        False  False        False     False        False    False  False   

     Assists  Shots  Shots On Target  ...  Fouls  Y

In [4]:
import numpy as np

# 1. Normalização por 90 minutos — evitando divisão por 0
df['Desarmes por 90m'] = df['Tackles'] / df['Minutes'].replace(0, np.nan) * 90
df['Intercept por 90m'] = df['Interceptions'] / df['Minutes'].replace(0, np.nan) * 90
df['Bolas zeradas po 90m'] = df['Clearances'] / df['Minutes'].replace(0, np.nan) * 90
df['Bloqueios por 90m'] = df['Blocks'] / df['Minutes'].replace(0, np.nan) * 90
df['Duelos no chão por 90m'] = df['Ground Duels'] / df['Minutes'].replace(0, np.nan) * 90
df['Duelos aereos por 90m'] = df['Aerial Duels'] / df['Minutes'].replace(0, np.nan) * 90

# 2. Metrica para versatilidade dos defensores
df['Versatilidade defensiva'] = np.where(
    df['Aerial Duels'] == 0,
    np.nan,
    df['Ground Duels'] / df['Aerial Duels']
)

# 3. Eficiência nos desarmes %
df['Desarmes sucesso %'] = df['Tackles'] / (df['Tackles'] + df['Fouls']).replace(0, np.nan)

# 4. Envolvimento na qualidade da posse de bola
df['Impacto de Posse'] = np.where(
    df['Dispossessed'] == 0,
    np.nan,
    df['Possession Won'] / df['Dispossessed']
)


In [5]:
# NORMALIZAÇÃO DE DADOS DE MEIO-CAMPISTAS
# Normalização de dados por 90 minutos
df['Passes por 90m'] = np.where(
    (df['Minutes'] == 0) | pd.isna(df['Minutes']),
    np.nan,
    df['Passes'] / df['Minutes'] * 90
)

df['Quebra de linha por 90m'] = np.where(
    (df['Minutes'] == 0) | pd.isna(df['Minutes']),
    np.nan,
    df['Through Balls'] / df['Minutes'] * 90
)

df['Condução por 90m'] = np.where(
    (df['Minutes'] == 0) | pd.isna(df['Minutes']),
    np.nan,
    df['Carries'] / df['Minutes'] * 90
)

df['Condução ao ataque por 90m'] = np.where(
    (df['Minutes'] == 0) | pd.isna(df['Minutes']),
    np.nan,
    df['Progressive Carries'] / df['Minutes'] * 90
)

df['Assistencias por 90m'] = np.where(
    (df['Minutes'] == 0) | pd.isna(df['Minutes']),
    np.nan,
    df['Assists'] / df['Minutes'] * 90
)

df['Conduções que geraram assistencia por 90m'] = np.where(
    pd.isna(df['Condução ao ataque por 90m']) | pd.isna(df['Assistencias por 90m']),
    np.nan,
    df['Condução ao ataque por 90m'] + df['Assistencias por 90m']
)

# Eficácia técnica
df['Eficacia com a bola'] = np.where(
    (df['Dispossessed'] == 0) | pd.isna(df['Dispossessed']) | pd.isna(df['Possession Won']),
    np.nan,
    df['Possession Won'] / df['Dispossessed']
)

In [6]:
# NORMALIZAÇÃO DE DADOS DE ATACANTES
# Dados por 90m
df['Gols por 90m'] = np.where(
    (df['Minutes'] == 0) | pd.isna(df['Minutes']),
    np.nan,
    df['Goals'] / df['Minutes'] * 90
)

df['Chutes por 90m'] = np.where(
    (df['Minutes'] == 0) | pd.isna(df['Minutes']),
    np.nan,
    df['Shots'] / df['Minutes'] * 90
)

df['Chutes ao gol por 90m'] = np.where(
    (df['Minutes'] == 0) | pd.isna(df['Minutes']),
    np.nan,
    df['Shots On Target'] / df['Minutes'] * 90
)

# Eficácia no ataque
df['Grandes chances perdidas por 90m'] = np.where(
    (df['Minutes'] == 0) | pd.isna(df['Minutes']),
    np.nan,
    df['Big Chances Missed'] / df['Minutes'] * 90
)

df['Gols por Assistência'] = np.where(
    (df['Assists'] == 0) | pd.isna(df['Assists']) | pd.isna(df['Goals']),
    np.nan,
    df['Goals'] / df['Assists']
)

In [8]:
# confirmando que os dados foram adicionados
df.columns

Index(['Player Name', 'Club', 'Nationality', 'Position', 'Appearances',
       'Minutes', 'Goals', 'Assists', 'Shots', 'Shots On Target',
       'Conversion %', 'Big Chances Missed', 'Hit Woodwork', 'Offsides',
       'Touches', 'Passes', 'Successful Passes', 'Passes%', 'Crosses',
       'Successful Crosses', 'Crosses %', 'fThird Passes',
       'Successful fThird Passes', 'fThird Passes %', 'Through Balls',
       'Carries', 'Progressive Carries', 'Carries Ended with Goal',
       'Carries Ended with Assist', 'Carries Ended with Shot',
       'Carries Ended with Chance', 'Possession Won', 'Dispossessed',
       'Clean Sheets', 'Clearances', 'Interceptions', 'Blocks', 'Tackles',
       'Ground Duels', 'gDuels Won', 'gDuels %', 'Aerial Duels', 'aDuels Won',
       'aDuels %', 'Goals Conceded', 'xGoT Conceded', 'Own Goals', 'Fouls',
       'Yellow Cards', 'Red Cards', 'Saves', 'Saves %', 'Penalties Saved',
       'Clearances Off Line', 'Punches', 'High Claims', 'Goals Prevented',
       

In [19]:
df.to_csv('DF_ALL_Teans.csv', index=False)

# separando o df removendo jogadores do clube
df_clubes = df["Club"].value_counts() 
df_clubes

# separando os jogadores do southampton em um df separado
southampton_df = df[df["Club"] == "Southampton"]
southampton_df.to_csv("Southampton.csv", index=False)

# separando mais um df agora sem os jogadores do southampton
premier_league_df = df[df["Club"] != "Southampton"]
premier_league_df.to_csv("PL_data.csv", index=False)